In [1]:
import json
import urllib.request

In [2]:
BEACON_URL = "https://beacon.nist.gov/beacon/2.0/pulse/last"

In [3]:
# CODE BASE

def fetch_latest_pulse_int():
    """Fetch latest NIST beacon pulse outputValue as a 512-bit int."""
    req = urllib.request.Request(
        BEACON_URL,
        headers={"Accept": "application/json"}
    )
    with urllib.request.urlopen(req, timeout=10) as r:
        data = r.read().decode("utf-8")
    obj = json.loads(data)
    pulse = obj["pulse"]
    hex_out = pulse.get("outputValue")
    if not hex_out:
        raise ValueError("Pulse missing outputValue")
    return int(hex_out, 16)


class SinglePulseBitPool:
    """A fixed 512-bit pool. No refills allowed."""
    def __init__(self, pulse_int_512):
        self.pool = pulse_int_512
        self.pool_bits = 512

    def get_bits(self, k):
        if k <= 0:
            return 0
        if k > self.pool_bits:
            raise RuntimeError(
                f"Single pulse exhausted: need {k} more bits, "
                f"only {self.pool_bits} left."
            )
        mask = (1 << k) - 1
        val = self.pool & mask
        self.pool >>= k
        self.pool_bits -= k
        return val


def bits_needed_for_span(span: int) -> int:
    """Ceiling log2(span), with exact handling for powers of two."""
    if span <= 1:
        return 0
    bl = span.bit_length()
    if span == 1 << (bl - 1):  # exact power of two
        return bl - 1
    return bl


def quantum_randint(low:int, high:int, n_samples:int=1):
    """
    Return uniform quantum random integer(s) in [low, high] from ONE pulse only.

    Raises ValueError upfront if the *minimum required bits*
    n_samples * ceil(log2(high-low+1)) exceeds 512.

    Also may raise RuntimeError if rejection sampling needs extra bits
    beyond that minimum and the single pulse is depleted.
    """
    if high < low:
        raise ValueError("high must be >= low")
    if n_samples < 1:
        raise ValueError("n_samples must be >= 1")

    span = high - low + 1
    if span == 1:
        return low if n_samples == 1 else [low] * n_samples

    bits_per_sample = bits_needed_for_span(span)
    min_bits_total = bits_per_sample * n_samples

    if min_bits_total > 512:
        raise ValueError(
            f"Request needs at least {min_bits_total} bits > 512-bit single pulse. "
            f"(bits/sample={bits_per_sample}, n_samples={n_samples}, span={span})"
        )

    pulse_int = fetch_latest_pulse_int()
    pool = SinglePulseBitPool(pulse_int)

    def one_sample():
        k = bits_per_sample
        # exact uniform via rejection on k bits
        while True:
            u = pool.get_bits(k)  # may raise RuntimeError if pulse exhausted
            if u < span:
                return low + u

    if n_samples == 1:
        return one_sample()
    return [one_sample() for _ in range(n_samples)]

In [4]:
# FUNCTIONALITY TEST
print("coin flips:", quantum_randint(0, 1, n_samples=8))
print("dice:", quantum_randint(1, 6, n_samples=5))
print("int64:", quantum_randint(-(1 << 63), (1 << 63) - 1))


coin flips: [0, 1, 1, 1, 0, 0, 1, 0]
dice: [2, 6, 6, 3, 5]
int64: 3356294498766908238
